In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4', temperature=TEMPERATURE)
llm4_32k = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model, which is one of the most advanced language models available. Specifically, I am running on version 3.5.0 of the GPT-3 model. This version was released on March 1, 2023. Is there anything specific you would like to know about my model version?"

In [5]:
llm3_5.model_name, llm4.model_name, llm4_32k.model_name

('gpt-3.5-turbo-16k', 'gpt-4', 'gpt-4-32k')

In [7]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=2,
    max_tokens=10000,
    check_final=True
)

primary_model = 4
sage_model = None

if primary_model == 4:
    primary_model = LangChainLLM(llm4, logger=logger)
    sage_llm = primary_model
else:
    primary_model = LangChainLLM(llm3_5, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000)
    sage_model = LangChainLLM(llm4, logger=logger, long_context_llm=llm4_32k, long_context_token_threshold=3000) \
        if sage_model is None else primary_model
executor = Orchestrator(
    config=config,
    llm=primary_model,
    prompts=prompts,
    markdown_logger=logger,
    sage_llm=sage_model,
)

experiment_name = 'example'
# executor.start("""What's x mod (y - z) where x = 10, y = 7, z = 4.""")
executor.start("""
Solve this 4x4 Sudoku.

```text
4 3 _ 1
1 _ 4 3
2 _ 3 _
3 4 _ 2
```

(`_` represents an empty cell.)

Solve in your head. Do NOT use Python programming.
""")

Pause: Pause after initial solving expansion

In [8]:
# executor._prompts = PromptDb.load_defaults()

In [9]:
# executor._chain = executor._chain[:-1]
# executor._chain[-1]

In [10]:
# executor.chain[-1].description

In [11]:
executor.resume(10000, additional_tokens_for_smarter_llm=0, unblock_initial_expansion=True)

In [12]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
executor.log_configs(logger)
logger.log_conversation(executor.show_conversation_thread(with_header=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")
